Check these out later: https://www.tutorialspoint.com/natural_language_toolkit/natural_language_toolkit_synonym_antonym_replacement.htm, https://www.kaggle.com/code/rohitsingh9990/data-augmentation-by-synonym-replacement

In [1]:
import pandas as pd
import numpy as np
import os


DATA_PATH = os.path.join("..", "data")

In [2]:
train_df = pd.read_csv(os.path.join(DATA_PATH, "train_essays.csv"))
train_df

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [3]:
train_df[train_df.generated == 1]

,id,prompt_id,text,generated
704,82131f68,1,"This essay will analyze, discuss and prove one...",1
740,86fe4f18,1,I strongly believe that the Electoral College ...,1
1262,eafb8a56,0,"Limiting car use causes pollution, increases c...",1


In [4]:
def read_gpt(file: str) -> pd.Series:
    with open(os.path.join(DATA_PATH, file)) as f:
        contents = f.read()
    rows = contents.split("~")
    return pd.Series(rows)


chat_gpt_environment = read_gpt("chatgpt_environment.txt")
envir_df = pd.DataFrame({"text": chat_gpt_environment, 
                         "prompt_id": np.zeros_like(chat_gpt_environment),
                         "generated": np.ones_like(chat_gpt_environment)})

chat_gpt_electoral = read_gpt("chatgpt_electoral.txt")
elect_df = pd.DataFrame({"text": chat_gpt_electoral, 
                         "prompt_id": np.ones_like(chat_gpt_electoral),
                         "generated": np.ones_like(chat_gpt_electoral)})

train_df.drop("id", axis=1, inplace=True)
pd.concat([envir_df, elect_df, train_df])

,text,prompt_id,generated
0,,0,1
1,"In recent years, there has been a growing con...",0,1
2,In the face of escalating environmental conce...,0,1
3,In the pursuit of a sustainable and environme...,0,1
4,In the face of pressing environmental challen...,0,1
...,...,...,...
1373,There has been a fuss about the Elector Colleg...,1,0
1374,Limiting car usage has many advantages. Such a...,0,0
1375,There's a new trend that has been developing f...,0,0
1376,As we all know cars are a big part of our soci...,0,0
